In [1]:
import os

img_test_dir = "img_test"

In [2]:
# check if dir is there
if os.path.exists(img_test_dir):
    print("Path exists")
else:
    print("nope")

Path exists


In [3]:
label_dict = {
    "rotten banana": 0,
    "rotten apple": 1,
    "fresh banana": 2,
    "fresh mango": 3,
    "fresh orange": 4,
    "fresh apple": 5,
    "rotten mango": 6,
    "rotten orange": 7
}

label_dict_no_ws = {
    "rotten_banana": 0,
    "rotten_apple": 1,
    "fresh_banana": 2,
    "fresh_mango": 3,
    "fresh_orange": 4,
    "fresh_apple": 5,
    "rotten_mango": 6,
    "rotten_orange": 7
}

In [4]:
labels = list(label_dict.keys())
labels

['rotten orange',
 'rotten apple',
 'rotten mango',
 'fresh banana',
 'rotten banana',
 'fresh apple',
 'fresh orange',
 'fresh mango']

In [5]:
# eliminate the whitespace in labels
labels_no_whitespace = []
for label in labels:
    labels_no_whitespace.append(label.replace(" ", "_"))

In [6]:
labels_no_whitespace

['rotten_orange',
 'rotten_apple',
 'rotten_mango',
 'fresh_banana',
 'rotten_banana',
 'fresh_apple',
 'fresh_orange',
 'fresh_mango']

In [7]:
def get_ground_truth(path):
    images = os.listdir(path)
    print("Total {} images in {}".format(len(images), path))
    
    ground_truth_dict = {}
    
    for img in images:
        for label in labels_no_whitespace:
            if label in img:
                img_path = os.path.join(img_test_dir, img)
                ground_truth_dict[img_path] = label_dict_no_ws[label]
                
    return ground_truth_dict

In [8]:
ground_truth = get_ground_truth(img_test_dir)

Total 1369 images in img_test


In [9]:
# pickle the ground_truths
import pickle

pickle_dir = "inception_pickled"
pickle_out_file = os.path.join(pickle_dir, "ground_truth.pickle")

gt_pickle_out = open(pickle_out_file, "wb")
pickle.dump(ground_truth, gt_pickle_out)

In [10]:
# check if pickled properly
pickle_in = pickle.load(open(pickle_out_file, "rb"))
print(pickle_in)

{'img_test/rotten_mango_162.jpg': 6, 'img_test/fresh_apple_111.jpeg': 5, 'img_test/fresh_apple_198.jpeg': 5, 'img_test/rotten_apple_234.jpg': 1, 'img_test/fresh_apple_16.jpeg': 5, 'img_test/fresh_apple_112.jpeg': 5, 'img_test/rotten_banana_26.jpg': 0, 'img_test/rotten_apple_123.jpg': 1, 'img_test/rotten_banana_151.jpg': 0, 'img_test/fresh_mango_88.jpg': 3, 'img_test/rotten_orange_21.jpeg': 7, 'img_test/fresh_mango_48.jpg': 3, 'img_test/fresh_apple_74.jpeg': 5, 'img_test/fresh_apple_36.jpeg': 5, 'img_test/rotten_mango_104.jpg': 6, 'img_test/rotten_orange_122.jpg': 7, 'img_test/rotten_banana_225.jpg': 0, 'img_test/fresh_apple_121.jpeg': 5, 'img_test/rotten_apple_90.jpg': 1, 'img_test/fresh_orange_59.jpg': 4, 'img_test/rotten_mango_9.jpg': 6, 'img_test/rotten_mango_143.jpg': 6, 'img_test/fresh_orange_78.jpeg': 4, 'img_test/fresh_apple_299.jpeg': 5, 'img_test/fresh_apple_175.jpeg': 5, 'img_test/rotten_orange_36.jpeg': 7, 'img_test/fresh_mango_97.jpg': 3, 'img_test/fresh_apple_162.jpeg': 5,

In [11]:
# now for classification

import tensorflow as tf

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

def classify(image_path):
    # Read in the image_data
    image_data = tf.gfile.FastGFile(image_path, 'rb').read()

    # Loads label file, strips off carriage return
    label_lines = [line.rstrip() for line
                   in tf.gfile.GFile("retrained_labels.txt")]

    # label_names = label_lines

    # Unpersists graph from file
    with tf.gfile.FastGFile("retrained_graph.pb", 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        tf.import_graph_def(graph_def, name='')

    with tf.Session() as sess:
        # Feed the image_data as input to the graph and get first prediction
        softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')

        predictions = sess.run(softmax_tensor, \
                               {'DecodeJpeg/contents:0': image_data})

        # global_pred = predictions

        # Sort to show labels of first prediction in order of confidence
        top_k = predictions[0].argsort()[-len(predictions[0]):][::-1]
        # label_id = top_k

        
        print('Processing {} ...................DONE'.format(image_path))
        resultDict = {}
        for node_id in top_k:
            human_string = label_lines[node_id]
            score = predictions[0][node_id]
#              print('Category : {} \nScore = {} %'.format(human_string, score * 100))

            resultDict[human_string] = score
        
        #print('Category : {}\tScore : {}'.format(label_lines[node_id], predictions[0][node_id]))

        #print('\nDONE===========================', end='\n')
        
        return resultDict


In [12]:
image_dir = img_test_dir

images = os.listdir(image_dir)
label_lines = [line.rstrip() for line
                   in tf.gfile.GFile("retrained_labels.txt")]
pred_dict = {}
    
# create graph
# with tf.gfile.FastGFile("retrained_graph.pb", 'rb') as f:
#     graph_def = tf.GraphDef()
#     graph_def.ParseFromString(f.read())
#     tf.import_graph_def(graph_def, name='')
    
def create_graph():
  """Creates a graph from saved GraphDef file and returns a saver."""
  # Creates graph from saved graph_def.pb.
  with tf.gfile.FastGFile("retrained_graph.pb", 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    _ = tf.import_graph_def(graph_def, name='')


In [13]:
# def run_inference(img_dir):
#     create_graph()
    
#     pred_dict = {}
    
#     for image in images:
#         image = os.path.join(image_dir, image)
#         image_data = tf.gfile.FastGFile(image, 'rb').read()
        
#         with tf.Session() as sess:
#             # Feed the image_data as input to the graph and get first prediction
#             softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')

#             predictions = sess.run(softmax_tensor, \
#                                     {'DecodeJpeg/contents:0': image_data})


#             # Sort to show labels of first prediction in order of confidence
#             top_k = predictions[0].argsort()[-len(predictions[0]):][::-1]
#             # label_id = top_k

        
#             print('Processing {} ...................DONE'.format(image))
#             resultDict = {}
#             for node_id in top_k:
#                 human_string = label_lines[node_id]
#                 score = predictions[0][node_id]
                
#                 resultDict[human_string] = score
                
#         # get max score
#         maxKey = max(rDict.keys(), key=(lambda k: rDict[k]))
#         pred_dict[image] = label_dict_no_ws[maxKey]
        
#         return pred_dict

In [14]:
def run_inference(img_dir):
    create_graph()
    
    pred_dict = {}
        
    with tf.Session() as sess:
        softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')
        for image in images:
            image = os.path.join(image_dir, image)
            image_data = tf.gfile.FastGFile(image, 'rb').read()
                # Feed the image_data as input to the graph and get first prediction
            

            predictions = sess.run(softmax_tensor, \
                                        {'DecodeJpeg/contents:0': image_data})


                # Sort to show labels of first prediction in order of confidence
            top_k = predictions[0].argsort()[-len(predictions[0]):][::-1]
                # label_id = top_k

        
            print('Processing {} ...................DONE'.format(image))
            resultDict = {}
            for node_id in top_k:
                human_string = label_lines[node_id]
                score = predictions[0][node_id]
                
                resultDict[human_string] = score
                
            # get max score
            maxKey = max(resultDict.keys(), key=(lambda k: resultDict[k]))
            pred_dict[image] = label_dict[maxKey]
        
    return pred_dict

In [15]:
d = run_inference(img_dir=img_test_dir)

Processing img_test/fresh_apple_65.jpeg ...................DONE
Processing img_test/rotten_banana_250.jpg ...................DONE
Processing img_test/fresh_apple_50.jpeg ...................DONE
Processing img_test/fresh_apple_51.jpeg ...................DONE
Processing img_test/fresh_apple_52.jpeg ...................DONE
Processing img_test/fresh_apple_53.jpeg ...................DONE
Processing img_test/fresh_apple_54.jpeg ...................DONE
Processing img_test/fresh_apple_55.jpeg ...................DONE
Processing img_test/fresh_apple_56.jpeg ...................DONE
Processing img_test/fresh_apple_57.jpeg ...................DONE
Processing img_test/fresh_apple_58.jpeg ...................DONE
Processing img_test/fresh_apple_59.jpeg ...................DONE
Processing img_test/fresh_apple_6.jpeg ...................DONE
Processing img_test/fresh_apple_60.jpeg ...................DONE
Processing img_test/fresh_apple_61.jpeg ...................DONE
Processing img_test/fresh_apple_62.jpeg

Processing img_test/fresh_apple_172.jpeg ...................DONE
Processing img_test/fresh_apple_173.jpeg ...................DONE
Processing img_test/fresh_apple_175.jpeg ...................DONE
Processing img_test/fresh_apple_176.jpeg ...................DONE
Processing img_test/fresh_apple_177.jpeg ...................DONE
Processing img_test/fresh_apple_178.jpeg ...................DONE
Processing img_test/fresh_apple_179.jpeg ...................DONE
Processing img_test/fresh_apple_18.jpeg ...................DONE
Processing img_test/fresh_apple_180.jpeg ...................DONE
Processing img_test/fresh_apple_181.jpeg ...................DONE
Processing img_test/fresh_apple_182.jpeg ...................DONE
Processing img_test/fresh_apple_183.jpeg ...................DONE
Processing img_test/fresh_apple_184.jpeg ...................DONE
Processing img_test/fresh_apple_185.jpeg ...................DONE
Processing img_test/fresh_apple_186.jpeg ...................DONE
Processing img_test/fresh_

Processing img_test/fresh_apple_300.jfif ...................DONE
Processing img_test/fresh_apple_301.jfif ...................DONE
Processing img_test/fresh_apple_302.jfif ...................DONE
Processing img_test/fresh_apple_303.jfif ...................DONE
Processing img_test/fresh_apple_304.jfif ...................DONE
Processing img_test/fresh_apple_305.jfif ...................DONE
Processing img_test/fresh_apple_306.jfif ...................DONE
Processing img_test/fresh_apple_307.jpg ...................DONE
Processing img_test/fresh_apple_308.jpeg ...................DONE
Processing img_test/fresh_apple_309.jpeg ...................DONE
Processing img_test/fresh_apple_31.jpeg ...................DONE
Processing img_test/fresh_apple_310.jpeg ...................DONE
Processing img_test/fresh_apple_311.jpeg ...................DONE
Processing img_test/fresh_apple_312.jpeg ...................DONE
Processing img_test/fresh_apple_313.jpeg ...................DONE
Processing img_test/fresh_a

Processing img_test/fresh_orange_65.jpg ...................DONE
Processing img_test/fresh_orange_66.jpg ...................DONE
Processing img_test/fresh_orange_67.jpg ...................DONE
Processing img_test/fresh_orange_68.jpg ...................DONE
Processing img_test/fresh_orange_69.jpg ...................DONE
Processing img_test/fresh_orange_7.jpeg ...................DONE
Processing img_test/fresh_orange_70.jpg ...................DONE
Processing img_test/fresh_orange_71.jpg ...................DONE
Processing img_test/fresh_orange_73.jpeg ...................DONE
Processing img_test/fresh_orange_74.jpeg ...................DONE
Processing img_test/fresh_orange_75.jpeg ...................DONE
Processing img_test/fresh_orange_76.jpeg ...................DONE
Processing img_test/fresh_orange_77.jpeg ...................DONE
Processing img_test/fresh_orange_78.jpeg ...................DONE
Processing img_test/fresh_orange_79.jpg ...................DONE
Processing img_test/fresh_orange_8

Processing img_test/rotten_orange_64.jpeg ...................DONE
Processing img_test/rotten_orange_64.jpg ...................DONE
Processing img_test/rotten_orange_65.jpeg ...................DONE
Processing img_test/rotten_orange_65.jpg ...................DONE
Processing img_test/rotten_orange_66.jpeg ...................DONE
Processing img_test/rotten_orange_66.jpg ...................DONE
Processing img_test/rotten_orange_67.jpeg ...................DONE
Processing img_test/rotten_orange_67.jpg ...................DONE
Processing img_test/rotten_orange_68.jpeg ...................DONE
Processing img_test/rotten_orange_68.jpg ...................DONE
Processing img_test/rotten_orange_69.jpeg ...................DONE
Processing img_test/rotten_orange_69.jpg ...................DONE
Processing img_test/rotten_orange_7.jpeg ...................DONE
Processing img_test/rotten_orange_7.jpg ...................DONE
Processing img_test/rotten_orange_71.jpg ...................DONE
Processing img_test/

...

Processing img_test/rotten_banana_241.jpg ...................DONE
Processing img_test/rotten_banana_242.jpg ...................DONE
Processing img_test/rotten_banana_243.jpg ...................DONE
Processing img_test/rotten_banana_244.jpg ...................DONE
Processing img_test/rotten_banana_245.jpg ...................DONE
Processing img_test/rotten_banana_246.jpg ...................DONE
Processing img_test/rotten_banana_247.jpg ...................DONE
Processing img_test/rotten_banana_248.jpg ...................DONE
Processing img_test/rotten_banana_249.jpg ...................DONE
Processing img_test/rotten_banana_25.jpg ...................DONE
Processing img_test/rotten_banana_251.jpg ...................DONE
Processing img_test/rotten_banana_252.jpg ...................DONE
Processing img_test/rotten_banana_253.jpg ...................DONE
Processing img_test/rotten_banana_254.jpg ...................DONE
Processing img_test/rotten_banana_255.jpg ...................DONE
Processing 

Processing img_test/rotten_banana_a.jpeg ...................DONE
Processing img_test/rotten_mango_128.jpg ...................DONE
Processing img_test/rotten_mango_143.jpg ...................DONE
Processing img_test/rotten_mango_162.jpg ...................DONE
Processing img_test/rotten_mango_190.jpg ...................DONE
Processing img_test/rotten_mango_229.jpg ...................DONE
Processing img_test/rotten_mango_244.jpg ...................DONE
Processing img_test/rotten_mango_268.jpg ...................DONE
Processing img_test/rotten_mango_3.jpg ...................DONE
Processing img_test/rotten_mango_315.jpg ...................DONE
Processing img_test/rotten_mango_49.jpg ...................DONE
Processing img_test/rotten_orange_1.jpg ...................DONE
Processing img_test/rotten_orange_12.jpeg ...................DONE
Processing img_test/rotten_orange_17.jpg ...................DONE
Processing img_test/rotten_orange_27.jpeg ...................DONE
Processing img_test/rotten_

Processing img_test/rotten_mango_46.jpg ...................DONE
Processing img_test/rotten_mango_47.jpg ...................DONE
Processing img_test/rotten_mango_48.jpg ...................DONE
Processing img_test/rotten_orange_18.jpeg ...................DONE
Processing img_test/rotten_orange_18.jpg ...................DONE
Processing img_test/rotten_orange_19.jpeg ...................DONE
Processing img_test/rotten_orange_19.jpg ...................DONE
Processing img_test/rotten_orange_2.jpg ...................DONE
Processing img_test/rotten_orange_20.jpeg ...................DONE
Processing img_test/rotten_orange_21.jpeg ...................DONE
Processing img_test/rotten_orange_22.jpeg ...................DONE
Processing img_test/rotten_orange_23.jpeg ...................DONE
Processing img_test/rotten_orange_23.jpg ...................DONE
Processing img_test/rotten_orange_24.jpeg ...................DONE
Processing img_test/rotten_orange_24.jpg ...................DONE
Processing img_test/ro

Processing img_test/rotten_apple_233.jpg ...................DONE
Processing img_test/rotten_apple_234.jpg ...................DONE
Processing img_test/rotten_apple_235.jpg ...................DONE
Processing img_test/rotten_apple_236.jpg ...................DONE
Processing img_test/rotten_apple_237.jpg ...................DONE
Processing img_test/rotten_apple_24.jpg ...................DONE
Processing img_test/rotten_apple_242.jpg ...................DONE
Processing img_test/rotten_apple_244.jpg ...................DONE
Processing img_test/rotten_apple_246.jpg ...................DONE
Processing img_test/rotten_apple_249.jpg ...................DONE
Processing img_test/rotten_apple_251.jpg ...................DONE
Processing img_test/rotten_apple_256.jpg ...................DONE
Processing img_test/rotten_apple_26.jpg ...................DONE
Processing img_test/rotten_apple_262.jpg ...................DONE
Processing img_test/rotten_apple_263.jpg ...................DONE
Processing img_test/rotten_

Processing img_test/fresh_mango_112.jpg ...................DONE
Processing img_test/fresh_mango_113.jpg ...................DONE
Processing img_test/fresh_mango_114.jpg ...................DONE
Processing img_test/fresh_mango_116.jpg ...................DONE
Processing img_test/fresh_mango_117.jpg ...................DONE
Processing img_test/fresh_mango_118.jpg ...................DONE
Processing img_test/fresh_mango_119.jpg ...................DONE
Processing img_test/fresh_mango_121.jpg ...................DONE
Processing img_test/fresh_mango_21.jpg ...................DONE
Processing img_test/fresh_mango_25.jpg ...................DONE
Processing img_test/fresh_mango_26.jpg ...................DONE
Processing img_test/fresh_mango_27.jpg ...................DONE
Processing img_test/fresh_mango_28.jpg ...................DONE
Processing img_test/fresh_mango_29.jpg ...................DONE
Processing img_test/fresh_mango_31.jpg ...................DONE
Processing img_test/fresh_mango_33.jpg ........

In [16]:
result_dict = d

In [17]:
len(result_dict)

1369

In [18]:
# pickle again

result_pickle_file = os.path.join(pickle_dir, "inception_predition.pickle")
pickle_out = open(result_pickle_file, "wb")
pickle.dump(result_dict, pickle_out)

pickle_out.close()

In [19]:
# check again
pickle_load = pickle.load(open(result_pickle_file, "rb"))
print(pickle_load)

{'img_test/rotten_mango_162.jpg': 6, 'img_test/fresh_apple_111.jpeg': 5, 'img_test/fresh_apple_198.jpeg': 5, 'img_test/rotten_apple_234.jpg': 1, 'img_test/fresh_apple_16.jpeg': 5, 'img_test/fresh_apple_112.jpeg': 5, 'img_test/rotten_banana_26.jpg': 0, 'img_test/rotten_apple_123.jpg': 1, 'img_test/rotten_banana_151.jpg': 0, 'img_test/fresh_mango_88.jpg': 3, 'img_test/rotten_orange_21.jpeg': 1, 'img_test/fresh_mango_48.jpg': 3, 'img_test/fresh_apple_74.jpeg': 5, 'img_test/fresh_apple_36.jpeg': 5, 'img_test/rotten_mango_104.jpg': 6, 'img_test/rotten_orange_122.jpg': 7, 'img_test/rotten_banana_225.jpg': 0, 'img_test/fresh_apple_121.jpeg': 5, 'img_test/rotten_apple_90.jpg': 1, 'img_test/fresh_orange_59.jpg': 4, 'img_test/rotten_mango_9.jpg': 6, 'img_test/rotten_mango_143.jpg': 6, 'img_test/fresh_orange_78.jpeg': 4, 'img_test/fresh_apple_299.jpeg': 3, 'img_test/fresh_apple_175.jpeg': 5, 'img_test/rotten_orange_36.jpeg': 4, 'img_test/fresh_mango_97.jpg': 3, 'img_test/fresh_apple_162.jpeg': 5,